In [1]:
import catboost as cb
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

c:\Users\Administrator\Desktop\FinetuningOptuna\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("C:/Users/Administrator/Desktop/FinetuningOptuna/heart.csv")
df = pd.get_dummies(df)
X = df.drop("HeartDisease", axis=1).copy()
y = df["HeartDisease"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=369)


In [5]:
model = cb.CatBoostClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

Learning rate set to 0.008526
0:	learn: 0.6858647	total: 142ms	remaining: 2m 21s
1:	learn: 0.6807758	total: 144ms	remaining: 1m 11s
2:	learn: 0.6735780	total: 146ms	remaining: 48.5s
3:	learn: 0.6666494	total: 148ms	remaining: 36.7s
4:	learn: 0.6613959	total: 149ms	remaining: 29.6s
5:	learn: 0.6542955	total: 150ms	remaining: 24.9s
6:	learn: 0.6473621	total: 152ms	remaining: 21.6s
7:	learn: 0.6407391	total: 154ms	remaining: 19.1s
8:	learn: 0.6348275	total: 156ms	remaining: 17.2s
9:	learn: 0.6296076	total: 158ms	remaining: 15.6s
10:	learn: 0.6239510	total: 159ms	remaining: 14.3s
11:	learn: 0.6181484	total: 161ms	remaining: 13.3s
12:	learn: 0.6127407	total: 163ms	remaining: 12.4s
13:	learn: 0.6084606	total: 164ms	remaining: 11.6s
14:	learn: 0.6042713	total: 165ms	remaining: 10.8s
15:	learn: 0.5999619	total: 167ms	remaining: 10.2s
16:	learn: 0.5952038	total: 168ms	remaining: 9.74s
17:	learn: 0.5899774	total: 170ms	remaining: 9.28s
18:	learn: 0.5851817	total: 173ms	remaining: 8.92s
19:	learn

0.9166666666666666

In [10]:
def objective(trial):
    hiper = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'model_size_reg': trial.suggest_float('model_size_reg', 0, 1),
        'rsm': trial.suggest_float('rsm', 0.5, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 1, 20),
        #'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 10),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'one_hot_max_size': trial.suggest_int('one_hot_max_size', 2, 255),
        #'max_leaves': trial.suggest_int('max_leaves', 16, 64),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'score_function': trial.suggest_categorical('score_function', ['Cosine', 'L2']),
        #'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
        'leaf_estimation_method': trial.suggest_categorical('leaf_estimation_method', ['Newton', 'Gradient']),
        'verbose': False,
        'thread_count': -1,
        'eval_metric': 'Accuracy' 
    }
    
    if hiper['bootstrap_type'] == 'Bayesian':
        hiper.pop('subsample') 
    model = cb.CatBoostClassifier(**hiper)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_jobs=-1, n_trials=1000)
    

[I 2025-08-22 22:54:06,290] A new study created in memory with name: no-name-f0f5e82e-02f3-4d8d-b7af-e9c52cf23980
[I 2025-08-22 22:54:06,821] Trial 4 finished with value: 0.9094202898550725 and parameters: {'iterations': 157, 'learning_rate': 0.020780877879871, 'depth': 5, 'l2_leaf_reg': 4.745827617173099, 'model_size_reg': 0.17372137822614642, 'rsm': 0.9021195908566799, 'border_count': 171, 'random_strength': 18.747685159542335, 'bootstrap_type': 'Bayesian', 'subsample': 0.8551568563674319, 'one_hot_max_size': 189, 'min_data_in_leaf': 45, 'grow_policy': 'SymmetricTree', 'score_function': 'L2', 'leaf_estimation_method': 'Newton'}. Best is trial 4 with value: 0.9094202898550725.
[I 2025-08-22 22:54:06,913] Trial 10 finished with value: 0.9057971014492754 and parameters: {'iterations': 113, 'learning_rate': 0.061975190022930324, 'depth': 4, 'l2_leaf_reg': 4.532580004223861, 'model_size_reg': 0.613771256844461, 'rsm': 0.8356324889554889, 'border_count': 180, 'random_strength': 18.14610373

In [11]:
model = cb.CatBoostClassifier(**study.best_params)
print(study.best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc  = accuracy_score(y_test, y_pred)
print(acc)

{'iterations': 494, 'learning_rate': 0.012855783035140048, 'depth': 5, 'l2_leaf_reg': 9.606913408201745, 'model_size_reg': 0.39972184433009544, 'rsm': 0.8343028017354867, 'border_count': 247, 'random_strength': 13.516883994268062, 'bootstrap_type': 'MVS', 'subsample': 0.8331104122104344, 'one_hot_max_size': 243, 'min_data_in_leaf': 69, 'grow_policy': 'Depthwise', 'score_function': 'Cosine', 'leaf_estimation_method': 'Gradient'}
0:	learn: 0.6860334	total: 5.22ms	remaining: 2.57s
1:	learn: 0.6795089	total: 10ms	remaining: 2.46s
2:	learn: 0.6750308	total: 15.8ms	remaining: 2.58s
3:	learn: 0.6687865	total: 20.7ms	remaining: 2.53s
4:	learn: 0.6636969	total: 24.8ms	remaining: 2.43s
5:	learn: 0.6578984	total: 29.2ms	remaining: 2.37s
6:	learn: 0.6516299	total: 33.8ms	remaining: 2.35s
7:	learn: 0.6475818	total: 37.9ms	remaining: 2.3s
8:	learn: 0.6420540	total: 43.3ms	remaining: 2.33s
9:	learn: 0.6364602	total: 47.5ms	remaining: 2.3s
10:	learn: 0.6313914	total: 51.1ms	remaining: 2.24s
11:	learn: